In [1]:
cd "/suphys/aloe8475"

/suphys/aloe8475


In [2]:
cd "Documents/edamame"

/import/silo2/aloe8475/Documents/edamame


In [3]:
from scipy.io import loadmat, savemat
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import colors
import datetime
import networkx as nx
from edamame import *
from tqdm import tqdm_notebook
import os
import edamame.core.wires as wires
from random import choice
import warnings
from IPython.core.debugger import set_trace
import nct
import bct

#warnings.filterwarnings('ignore')

In [4]:
import pickle 
import _pickle as cPickle
import gzip
def compressed_pickle(obj, filename,protocol=-1):
    with gzip.open(filename, 'wb') as f:
        cPickle.dump(obj, f, protocol)

In [5]:
cd "/import/silo2/aloe8475/Documents/CODE/Analysis/Functional Connectivity/Functional Tasks/"

/import/silo2/aloe8475/Documents/CODE/Analysis/Functional Connectivity/Functional Tasks


In [21]:
name='networks_LinearTransformation.pkl'
print('Loading Networks')
file = open(name, 'rb')
#     [ASN300,cluster1,cluster2,cluster3,time_index,nodesList] = pickle.load(file)
#     [ASN300,cluster1,cluster2,cluster3] = pickle.load(file)
[ASN300] = pickle.load(file)


Loading Networks


In [8]:
onAmp=[[] for i in range(len(ASN300))]
shortestPath=[[None]*10 for i in range(len(ASN300))]
for i in tqdm(range(len(ASN300))):
    for j in range(len(ASN300[i])):
        temp=getFarthestPairing(ASN300[i][j]['adj_matrix'])
        shortestPath[i][j]=nx.shortest_path_length(ASN300[i][j]['G'],temp[0],temp[1])
        onAmp[i].append(shortestPath[i][j]/5) #need to justify divide by 5 or come up with some other way

In [10]:
init=[]
for i in range(len(onAmp)):
    for j in range(len(onAmp[i])):
        amp=onAmp[i][j]
        connectivity=connectivity__(wires_dict=ASN300[i][j])
        init.append(inputPacker(get_MC,connectivity,amp))

In [11]:
#Run this 10 times
result=[]
for i in tqdm(range(10)):
    result.append(para_run(get_MC,init,nCPU=10))

In [12]:
temp=np.vstack(result)

In [13]:
temp.T

array([[ 1.93052165,  1.93052207,  1.93052194, ...,  1.93052167,
         1.93052219,  1.93052218],
       [ 0.78985943,  0.78985883,  0.78985943, ...,  0.78985883,
         0.78985883,  0.78985883],
       [ 1.5164769 ,  1.5164769 ,  1.5164769 , ...,  1.51647529,
         1.5164769 ,  1.51647688],
       ...,
       [ 2.95252059,  9.01038807,  2.29161649, ...,  0.45699448,
        10.45494198,  1.99535969],
       [ 5.18165907,  5.76218429,  1.98580036, ...,  5.91794645,
         1.57068613,  1.04050955],
       [ 1.04687877,  0.6421672 ,  5.01931968, ...,  1.39921887,
         1.30315898,  0.31589003]])

In [14]:
result2=np.mean(temp.T,axis=1)

In [15]:
name= r'/import/silo2/aloe8475/Documents/CODE/Data/Functional Connectivity/results_MemoryCapacity_10Sims'
compressed_pickle([result2],name)

# Random & Grid Networks:

In [22]:
# name='WS_networks_LinearTransformation.pkl'
name='WS_Allnetworks_LinearTransformation.pkl'
print('Loading WS')
file = open(name, 'rb')
[WS_Grid,WS_Random] = pickle.load(file)
print('Loaded')

Loading WS
Loaded


In [23]:
numRand=3
onAmp_Rand=[]
onAmp_Grid=[]
for i in range(len(WS_Random)):
    for j in range(numRand):
        onAmp_Rand.append(WS_Random[i][j]['sp_amp'])
        
onAmp_Grid.append(WS_Grid['sp_amp'])

In [27]:
len(onAmp_Rand)

900

In [121]:
initRand=[]
count=0
for i in tqdm(range(len(WS_Random))):
    for j in range(numRand):
        amp=onAmp_Rand[count]
        connectivity=connectivity__(graph=WS_Random[i][j]['G'])
        initRand.append(inputPacker(get_MC,connectivity,amp))
        count=count+1
        
initGrid=[]
amp=onAmp_Grid
connectivity=connectivity__(graph=WS_Grid['G'][0])
initGrid.append(inputPacker(get_MC,connectivity,amp))

Exception ignored in: <function WeakSet.__init__.<locals>._remove at 0x7f3a60c86e18>
Traceback (most recent call last):
  File "/suphys/aloe8475/anaconda3/lib/python3.7/_weakrefset.py", line 38, in _remove
    def _remove(item, selfref=ref(self)):
KeyboardInterrupt


In [32]:
#Run this 10 times
resultRand=[]
resultGrid=[]
for i in tqdm(range(10)):
    resultRand.append(para_run(get_MC,initRand,nCPU=10))
    resultGrid.append(para_run(get_MC,initGrid,nCPU=10))

In [103]:
temp=np.vstack(resultRand)
# resultRandtemp=np.mean(temp,axis=0)
resultRand2=np.mean(temp.T,axis=1)

In [119]:
resultRand2=resultRand2.reshape(300,3)

In [111]:
resultGrid2=np.mean(resultGrid)

In [120]:
name= r'/import/silo2/aloe8475/Documents/CODE/Data/Functional Connectivity/resultsRand_MemoryCapacity_10Sims'
compressed_pickle([resultRand2,resultGrid2],name)

## Barabasi-Albert Scale Free Networks

In [10]:
# name='WS_networks_LinearTransformation.pkl'
name='BA_Allnetworks_LinearTransformation.pkl'
print('Loading BA')
file = open(name, 'rb')
[BA_scaleFree] = pickle.load(file)
print('Loaded')

Loading BA
Loaded


In [11]:
onAmp_BA=[]
for i in range(len(BA_scaleFree)):
    onAmp_BA.append(BA_scaleFree[i]['sp_amp'])
        


In [12]:
initBA=[]
count=0
for i in tqdm(range(len(BA_scaleFree))):
    amp=onAmp_BA[count]
    connectivity=connectivity__(graph=BA_scaleFree[i]['G'])
    initBA.append(inputPacker(get_MC,connectivity,amp))
    count=count+1


In [13]:
#Run this 10 times
resultBA=[]
for i in tqdm(range(10)):
    resultBA.append(para_run(get_MC,initBA,nCPU=10))

In [14]:
temp=np.vstack(resultBA)
resultBA2=np.mean(temp.T,axis=1)

In [15]:
name= r'/import/silo2/aloe8475/Documents/CODE/Data/Functional Connectivity/resultsBA_MemoryCapacity_10Sims'
compressed_pickle([resultBA2],name)

## C. Elegans

In [129]:
name='elegans_LinearTransformation.pkl'
print('Loading Elegans Networks')
file = open(name, 'rb')
#     [ASN300,cluster1,cluster2,cluster3,time_index,nodesList] = pickle.load(file)
#     [ASN300,cluster1,cluster2,cluster3] = pickle.load(file)
[Elegans] = pickle.load(file)

print('Loaded')

Loading Elegans Networks
Loaded


In [130]:
#On Amp as a function of shortestPath
onAmp=[]
shortestPathElegans=[]
temp=getFarthestPairing(Elegans['adj_matrix'])
shortestPathElegans=nx.shortest_path_length(Elegans['G'],temp[0],temp[1])
onAmpElegans=shortestPathElegans/5

In [131]:
amp=onAmpElegans
connectivity=connectivity__(graph=Elegans['G'])

In [136]:
#Run this 10 times
resultElegans=[]
for i in tqdm(range(10)):
    resultElegans.append(get_MC(connectivity,amp))

In [137]:
temp=np.vstack(resultElegans)
resultElegans2=np.mean(temp.T,axis=1)

In [140]:
name= r'/import/silo2/aloe8475/Documents/CODE/Data/Functional Connectivity/resultsElegans_MemoryCapacity_10Sims'
compressed_pickle([resultElegans2],name)

## Bipartite

In [6]:
name='Barpartite_Allnetworks_LinearTransformation.pkl'
print('Loading BP Networks')
file = open(name, 'rb')
#     [ASN300,cluster1,cluster2,cluster3,time_index,nodesList] = pickle.load(file)
#     [ASN300,cluster1,cluster2,cluster3] = pickle.load(file)
[crossbar] = pickle.load(file)

print('Loaded')


Loading BP Networks
Loaded


In [7]:
temp=[]
shortestPath_crossbar=[]
onAmp_crossbar=[]

temp=[np.array([0,299])]#.append(getFarthestPairing(nx.adj_matrix(crossbar_graph).todense()))
shortestPath_crossbar=(nx.shortest_path_length(crossbar['G'],temp[0][0],temp[0][1]))
onAmp_crossbar=(shortestPath_crossbar/5)

In [8]:
amp=onAmp_crossbar
connectivity=connectivity__(graph=crossbar['G'])

In [9]:
#Run this 10 times
resultCrossbar=[]
for i in tqdm(range(10)):
    resultCrossbar.append(get_MC(connectivity,amp))

Exception ignored in: <generator object tqdm_notebook.__iter__ at 0x7f0eb6531e58>
Traceback (most recent call last):
  File "/suphys/aloe8475/anaconda3/lib/python3.7/site-packages/tqdm/_tqdm_notebook.py", line 226, in __iter__
    self.sp(bar_style='danger')
AttributeError: 'tqdm_notebook' object has no attribute 'sp'


2020-06-18 16:45:02,312:ERROR:Internal Python error in the inspect module.
Below is the traceback from this internal error.

2020-06-18 16:45:02,382:INFO:
Unfortunately, your original traceback can not be constructed.



Traceback (most recent call last):
  File "/suphys/aloe8475/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py", line 3296, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-9-99ad3a36013a>", line 4, in <module>
    resultCrossbar.append(get_MC(connectivity,amp))
  File "/import/silo2/aloe8475/Documents/edamame/edamame/analysis/benchmark.py", line 61, in get_MC
    findFirst=False, disable_tqdm=True)
  File "/import/silo2/aloe8475/Documents/edamame/edamame/core/simulate.py", line 190, in runSimulation
    this_realization = simulateNetwork(SimulationOptions, Connectivity, JunctionState, lite_mode, disable_tqdm, save_steps, **kwdict)
  File "/import/silo2/aloe8475/Documents/edamame/edamame/core/simulate.py", line 88, in simulateNetwork
    sol = simCore(connectivity, junctionState, this_stimulus, electrodes, simulationOptions.dt)
  File "/import/silo2/aloe8475/Documents/edamame/edamame/core/simulate.py", line 26, in simCore
    

KeyboardInterrupt: 

In [146]:
temp=np.vstack(resultCrossbar)
resultCrossbar2=np.mean(temp.T,axis=1)

In [147]:
name= r'/import/silo2/aloe8475/Documents/CODE/Data/Functional Connectivity/resultsBarpartite_MemoryCapacity_10Sims'
compressed_pickle([resultCrossbar2],name)